In [2]:
# ============================================================
# CELL X: NumPy compatibility patch for MAPS
# ============================================================
# MAPS uses np.Inf, which was removed in NumPy >= 2.0
# This patch restores it safely.

import numpy as np

if not hasattr(np, "Inf"):
    np.Inf = np.inf
    print("🩹 Patched np.Inf -> np.inf for MAPS compatibility")
else:
    print("✅ np.Inf already exists")


🩹 Patched np.Inf -> np.inf for MAPS compatibility


In [3]:
# ============================================================
# CELL 0: Clone original MAPS repository
# ============================================================

MAPS_GITHUB_URL = "https://github.com/mahmoodlab/MAPS.git"
MAPS_DIR = "/content/MAPS"

# Clone only if not already present
if not os.path.exists(MAPS_DIR):
    !git clone {MAPS_GITHUB_URL} {MAPS_DIR}
else:
    print("📦 MAPS repo already exists")

# Add MAPS to Python path
import sys
if MAPS_DIR not in sys.path:
    sys.path.insert(0, MAPS_DIR)

print("✅ MAPS repo ready and added to sys.path")


Cloning into '/content/MAPS'...
remote: Enumerating objects: 299, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 299 (delta 19), reused 43 (delta 7), pack-reused 232 (from 1)
Receiving objects: 100% (299/299), 157.54 MiB | 35.17 MiB/s, done.
Resolving deltas: 100% (95/95), done.
Updating files: 100% (137/137), done.
✅ MAPS repo ready and added to sys.path


In [1]:
# ============================================================
# CELL 1: Environment Setup
# ============================================================
# Purpose:
# - Mount Google Drive
# - Define project paths
# - Import core libraries
# - Sanity-check environment
#
# NOTE:
# - Do NOT do any pip installs in later cells.
# - If you install dependencies here, expect ONE runtime restart.
# ============================================================

# ---------- 1. Mount Google Drive ----------
from google.colab import drive
drive.mount('/content/drive')

# ---------- 2. Define project root ----------
# Adjust this path ONLY if your repo lives elsewhere
PROJECT_ROOT = "/content/drive/MyDrive/MAPS-update-main"

import os
assert os.path.exists(PROJECT_ROOT), (
    f"❌ Project root not found at {PROJECT_ROOT}\n"
    "Check the path or move the repo into Drive."
)

print(f"✅ Project root found: {PROJECT_ROOT}")

# ---------- 3. Define data paths ----------
DATA_PROCESSED = os.path.join(PROJECT_ROOT, "cHL_CODEX_processed")
DATA_SPATIAL   = os.path.join(PROJECT_ROOT, "cHL_CODEX_spatial_features")

assert os.path.exists(DATA_PROCESSED), "❌ cHL_CODEX_processed not found"
assert os.path.exists(DATA_SPATIAL),   "❌ cHL_CODEX_spatial_features not found"

print("✅ Data directories found")

# ---------- 4. Core imports ----------
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# ---------- 5. Torch check (for later MAPS training) ----------
import torch

print("✅ Core imports successful")
print(f"🖥️ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🖥️ GPU: {torch.cuda.get_device_name(0)}")

# ---------- 6. Global random seed (reproducibility) ----------
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print("🔒 Random seed set")
print("🚀 Cell 1 complete")


Mounted at /content/drive
✅ Project root found: /content/drive/MyDrive/MAPS-update-main
✅ Data directories found
✅ Core imports successful
🖥️ CUDA available: True
🖥️ GPU: Tesla T4
🔒 Random seed set
🚀 Cell 1 complete


In [5]:
# ============================================================
# CELL 2: Load datasets and sanity checks
# ============================================================
# Purpose:
# - Load processed spatial CSVs
# - Verify shapes and columns
# - Confirm required fields exist before KNN
# ============================================================

# ---------- 1. File paths ----------
TRAIN_PATH = os.path.join(DATA_SPATIAL, "train.csv")
VALID_PATH = os.path.join(DATA_SPATIAL, "valid.csv")
CLASS_NAMES_PATH = os.path.join(DATA_SPATIAL, "class_names.csv")

for p in [TRAIN_PATH, VALID_PATH, CLASS_NAMES_PATH]:
    assert os.path.exists(p), f"❌ Missing file: {p}"

print("✅ All CSV files found")

# ---------- 2. Load CSVs ----------
train_df = pd.read_csv(TRAIN_PATH)
valid_df = pd.read_csv(VALID_PATH)
class_names = pd.read_csv(CLASS_NAMES_PATH)

print(f"📊 Train shape: {train_df.shape}")
print(f"📊 Valid shape: {valid_df.shape}")
print(f"📊 Num classes: {len(class_names)}")

# ---------- 3. Required columns ----------
REQUIRED_COLS = [
    "X_cent",
    "Y_cent",
    "cell_label",
    "cellSize"
]

missing_train = [c for c in REQUIRED_COLS if c not in train_df.columns]
missing_valid = [c for c in REQUIRED_COLS if c not in valid_df.columns]

assert len(missing_train) == 0, f"❌ Train missing columns: {missing_train}"
assert len(missing_valid) == 0, f"❌ Valid missing columns: {missing_valid}"

print("✅ Required columns present")

# ---------- 4. Identify marker columns ----------
# Marker columns = everything except labels and spatial/meta fields
EXCLUDE_COLS = set([
    "X_cent",
    "Y_cent",
    "cell_label",
    "cellSize",
    "local_density",
    "distance_to_center"
])

marker_cols = [c for c in train_df.columns if c not in EXCLUDE_COLS]

print(f"🧬 Number of marker columns: {len(marker_cols)}")
print("🧬 Example markers:", marker_cols[:10])

# ---------- 5. Label sanity check ----------
num_unique_labels = train_df["cell_label"].nunique()
assert num_unique_labels == len(class_names), (
    f"❌ Label mismatch: train has {num_unique_labels}, "
    f"class_names has {len(class_names)}"
)

print("🏷️ Label sanity check passed")

# ---------- 6. Coordinate sanity check ----------
print("📍 Coordinate ranges (train):")
print("  X_cent:", train_df["X_cent"].min(), "→", train_df["X_cent"].max())
print("  Y_cent:", train_df["Y_cent"].min(), "→", train_df["Y_cent"].max())

# ---------- 7. Final confirmation ----------
print("🚀 Cell 2 complete: data loaded and validated")


✅ All CSV files found
📊 Train shape: (114984, 55)
📊 Valid shape: (28746, 55)
📊 Num classes: 16
✅ Required columns present
🧬 Number of marker columns: 49
🧬 Example markers: ['BCL.2', 'CCR6', 'CD11b', 'CD11c', 'CD15', 'CD16', 'CD162', 'CD163', 'CD2', 'CD20']
🏷️ Label sanity check passed
📍 Coordinate ranges (train):
  X_cent: 3 → 8082
  Y_cent: 2 → 8006
🚀 Cell 2 complete: data loaded and validated


In [6]:
# ============================================================
# CELL 3: Define feature groups
# ============================================================
# Purpose:
# - Explicitly define which features are used where
# - Freeze marker lists for reproducibility
# ============================================================

# ---------- 1. Core columns ----------
SPATIAL_COORDS = ["X_cent", "Y_cent"]
LABEL_COL = "cell_label"
META_COLS = ["cellSize"]

print("📌 Spatial coords:", SPATIAL_COORDS)
print("📌 Label column:", LABEL_COL)
print("📌 Meta columns:", META_COLS)

# ---------- 2. Marker columns (protein expression) ----------
# Exclude spatial, label, and engineered spatial features
EXCLUDE_COLS = set(SPATIAL_COORDS + META_COLS + [
    LABEL_COL,
    "local_density",
    "distance_to_center"
])

marker_cols = [c for c in train_df.columns if c not in EXCLUDE_COLS]

print(f"🧬 Total protein markers available: {len(marker_cols)}")

# ---------- 3. Select markers for KNN neighborhood summaries ----------
# We intentionally keep this small and biologically meaningful

KNN_MARKERS = [
    "CD4",     # Helper T cells
    "CD8",     # Cytotoxic T cells
    "CD20",    # B cells
    "CD68",    # Macrophages
    "CD163"    # M2 macrophages
]

# Verify all selected markers exist in the dataset
missing_knn_markers = [m for m in KNN_MARKERS if m not in marker_cols]
assert len(missing_knn_markers) == 0, (
    f"❌ Missing KNN markers: {missing_knn_markers}"
)

print("🧬 Markers used for KNN summaries:", KNN_MARKERS)

# ---------- 4. Feature accounting ----------
BASE_FEATURES = marker_cols + META_COLS + [
    "local_density",
    "distance_to_center"
]

print(f"📐 Baseline feature count (before KNN): {len(BASE_FEATURES)}")

# Planned KNN features
PLANNED_KNN_FEATURES = (
    ["knn_mean_dist", "knn_std_dist"] +
    [f"knn_mean_{m}" for m in KNN_MARKERS]
)

print("🧩 Planned KNN features:")
for f in PLANNED_KNN_FEATURES:
    print("  -", f)

print(f"📐 KNN feature count: {len(PLANNED_KNN_FEATURES)}")
print(f"📐 Total features after KNN: {len(BASE_FEATURES) + len(PLANNED_KNN_FEATURES)}")

# ---------- 5. Final confirmation ----------
print("🚀 Cell 3 complete: feature groups defined")


📌 Spatial coords: ['X_cent', 'Y_cent']
📌 Label column: cell_label
📌 Meta columns: ['cellSize']
🧬 Total protein markers available: 49
🧬 Markers used for KNN summaries: ['CD4', 'CD8', 'CD20', 'CD68', 'CD163']
📐 Baseline feature count (before KNN): 52
🧩 Planned KNN features:
  - knn_mean_dist
  - knn_std_dist
  - knn_mean_CD4
  - knn_mean_CD8
  - knn_mean_CD20
  - knn_mean_CD68
  - knn_mean_CD163
📐 KNN feature count: 7
📐 Total features after KNN: 59
🚀 Cell 3 complete: feature groups defined


In [7]:
# ============================================================
# CELL 4: Fit spatial KNN on TRAIN set
# ============================================================
# Purpose:
# - Build spatial neighborhoods using (X_cent, Y_cent)
# - TRAIN set only (no leakage)
# - Inspect distances for sanity before feature computation
# ============================================================

# ---------- 1. KNN parameters ----------
K = 5  # start simple, do NOT tune yet
print(f"🔢 Using K = {K} spatial neighbors")

# ---------- 2. Extract spatial coordinates (TRAIN) ----------
train_coords = train_df[SPATIAL_COORDS].values

print("📍 Train coordinates shape:", train_coords.shape)

# ---------- 3. Fit KNN ----------
# n_neighbors = K + 1 because the closest neighbor is the cell itself
knn = NearestNeighbors(
    n_neighbors=K + 1,
    metric="euclidean",
    algorithm="ball_tree"
)

knn.fit(train_coords)

# ---------- 4. Query neighbors ----------
distances, indices = knn.kneighbors(train_coords)

print("📐 Raw distance matrix shape:", distances.shape)
print("📐 Raw index matrix shape:", indices.shape)

# ---------- 5. Drop self-neighbor ----------
# distances[:, 0] == 0 (distance to itself)
train_knn_distances = distances[:, 1:]
train_knn_indices = indices[:, 1:]

print("📐 KNN distances (no self):", train_knn_distances.shape)
print("📐 KNN indices (no self):", train_knn_indices.shape)

# ---------- 6. Sanity checks ----------
# Check that no self-indices remain
assert not np.any(train_knn_indices == np.arange(len(train_df))[:, None]), \
    "❌ Self-neighbors still present"

# Check distance statistics
print("📊 Distance statistics (TRAIN):")
print("  Mean:", np.mean(train_knn_distances))
print("  Std :", np.std(train_knn_distances))
print("  Min :", np.min(train_knn_distances))
print("  Max :", np.max(train_knn_distances))

# ---------- 7. Inspect a few neighborhoods ----------
for i in [0, 1, 2]:
    print(f"\n🔍 Cell {i} neighbors:")
    print("  Indices :", train_knn_indices[i])
    print("  Distances:", np.round(train_knn_distances[i], 2))

# ---------- 8. Final confirmation ----------
print("🚀 Cell 4 complete: spatial KNN fitted on TRAIN set")


🔢 Using K = 5 spatial neighbors
📍 Train coordinates shape: (114984, 2)
📐 Raw distance matrix shape: (114984, 6)
📐 Raw index matrix shape: (114984, 6)
📐 KNN distances (no self): (114984, 5)
📐 KNN indices (no self): (114984, 5)
📊 Distance statistics (TRAIN):
  Mean: 22.714084337068496
  Std : 7.63694344975816
  Min : 5.0
  Max : 297.6390431378249

🔍 Cell 0 neighbors:
  Indices : [176 213 466 663 451]
  Distances: [18.97 23.09 26.63 32.06 35.23]

🔍 Cell 1 neighbors:
  Indices : [266 267 112 577 357]
  Distances: [11.66 14.42 23.35 27.66 29.68]

🔍 Cell 2 neighbors:
  Indices : [239 537 594 807 334]
  Distances: [15.26 26.25 36.24 40.52 42.06]
🚀 Cell 4 complete: spatial KNN fitted on TRAIN set


In [8]:
# ============================================================
# CELL 5: Compute label-free KNN features (TRAIN set)
# ============================================================
# Purpose:
# - Generate per-cell KNN spatial summary features
# - Label-free, safe for training
# - Append features to train_df
# ============================================================

# ---------- 1. Initialize feature containers ----------
n_cells = train_df.shape[0]

knn_mean_dist = np.zeros(n_cells)
knn_std_dist  = np.zeros(n_cells)

# One array per marker
knn_mean_marker = {
    m: np.zeros(n_cells) for m in KNN_MARKERS
}

# ---------- 2. Compute KNN distance features ----------
knn_mean_dist[:] = train_knn_distances.mean(axis=1)
knn_std_dist[:]  = train_knn_distances.std(axis=1)

# ---------- 3. Compute KNN marker-mean features ----------
for i in range(n_cells):
    neighbors = train_knn_indices[i]
    for m in KNN_MARKERS:
        knn_mean_marker[m][i] = train_df.loc[neighbors, m].mean()

# ---------- 4. Append features to DataFrame ----------
train_knn_df = train_df.copy()

train_knn_df["knn_mean_dist"] = knn_mean_dist
train_knn_df["knn_std_dist"]  = knn_std_dist

for m in KNN_MARKERS:
    train_knn_df[f"knn_mean_{m}"] = knn_mean_marker[m]

# ---------- 5. Sanity checks ----------
print("📐 Train shape before KNN:", train_df.shape)
print("📐 Train shape after  KNN:", train_knn_df.shape)

print("\n🧪 Example KNN features (first 5 rows):")
display(
    train_knn_df[
        ["knn_mean_dist", "knn_std_dist"] +
        [f"knn_mean_{m}" for m in KNN_MARKERS]
    ].head()
)

# ---------- 6. Final confirmation ----------
print("🚀 Cell 5 complete: KNN features computed for TRAIN set")


📐 Train shape before KNN: (114984, 55)
📐 Train shape after  KNN: (114984, 62)

🧪 Example KNN features (first 5 rows):


,knn_mean_dist,knn_std_dist,knn_mean_CD4,knn_mean_CD8,knn_mean_CD20,knn_mean_CD68,knn_mean_CD163
0,27.195556,5.882322,0.012218,0.052724,0.768365,0.000072,0.011370
1,21.353924,7.142123,0.273338,0.142740,0.003212,0.027929,0.020681
2,32.065914,10.048739,0.191928,0.002662,0.037227,0.042715,0.017929
3,21.366459,4.987426,0.064550,0.147240,0.000007,0.469820,0.212313
4,27.775427,9.062320,0.354901,0.033046,0.150913,0.154144,0.015737


🚀 Cell 5 complete: KNN features computed for TRAIN set


In [9]:
# ============================================================
# CELL 6: Compute label-free KNN features (VALID set)
# ============================================================
# Purpose:
# - Compute KNN features for validation data
# - KNN fit and query are done on VALID set only
# - Ensure column alignment with TRAIN KNN dataset
# ============================================================

# ---------- 1. Extract spatial coordinates (VALID) ----------
valid_coords = valid_df[SPATIAL_COORDS].values
print("📍 Valid coordinates shape:", valid_coords.shape)

# ---------- 2. Fit KNN on VALID set ----------
knn_valid = NearestNeighbors(
    n_neighbors=K + 1,   # self + K neighbors
    metric="euclidean",
    algorithm="ball_tree"
)

knn_valid.fit(valid_coords)

# ---------- 3. Query neighbors ----------
valid_distances, valid_indices = knn_valid.kneighbors(valid_coords)

# Drop self-neighbor
valid_knn_distances = valid_distances[:, 1:]
valid_knn_indices   = valid_indices[:, 1:]

print("📐 Valid KNN distances shape:", valid_knn_distances.shape)
print("📐 Valid KNN indices shape:", valid_knn_indices.shape)

# ---------- 4. Distance feature computation ----------
n_valid = valid_df.shape[0]

valid_knn_mean_dist = valid_knn_distances.mean(axis=1)
valid_knn_std_dist  = valid_knn_distances.std(axis=1)

# ---------- 5. Marker-mean feature computation ----------
valid_knn_mean_marker = {
    m: np.zeros(n_valid) for m in KNN_MARKERS
}

for i in range(n_valid):
    neighbors = valid_knn_indices[i]
    for m in KNN_MARKERS:
        valid_knn_mean_marker[m][i] = valid_df.loc[neighbors, m].mean()

# ---------- 6. Append features to VALID DataFrame ----------
valid_knn_df = valid_df.copy()

valid_knn_df["knn_mean_dist"] = valid_knn_mean_dist
valid_knn_df["knn_std_dist"]  = valid_knn_std_dist

for m in KNN_MARKERS:
    valid_knn_df[f"knn_mean_{m}"] = valid_knn_mean_marker[m]

# ---------- 7. Sanity checks ----------
print("📐 Valid shape before KNN:", valid_df.shape)
print("📐 Valid shape after  KNN:", valid_knn_df.shape)

print("\n🧪 Example VALID KNN features (first 5 rows):")
display(
    valid_knn_df[
        ["knn_mean_dist", "knn_std_dist"] +
        [f"knn_mean_{m}" for m in KNN_MARKERS]
    ].head()
)

# ---------- 8. Column alignment check ----------
assert list(train_knn_df.columns) == list(valid_knn_df.columns), (
    "❌ Train and Valid columns do not match after KNN feature generation"
)

print("✅ Train/Valid column alignment confirmed")
print("🚀 Cell 6 complete: KNN features computed for VALID set")


📍 Valid coordinates shape: (28746, 2)
📐 Valid KNN distances shape: (28746, 5)
📐 Valid KNN indices shape: (28746, 5)
📐 Valid shape before KNN: (28746, 55)
📐 Valid shape after  KNN: (28746, 62)

🧪 Example VALID KNN features (first 5 rows):


,knn_mean_dist,knn_std_dist,knn_mean_CD4,knn_mean_CD8,knn_mean_CD20,knn_mean_CD68,knn_mean_CD163
0,33.216251,20.636877,0.180109,0.014280,0.116580,0.105766,0.001879
1,42.482285,16.536490,0.165847,0.254602,0.191149,0.330677,0.170045
2,46.031657,3.080026,0.352765,0.074465,0.152807,0.244423,0.017929
3,25.853591,10.742058,0.326376,0.011663,0.329317,0.044188,0.021225
4,70.536415,3.288482,0.144776,0.163763,0.408329,0.167229,0.009162


✅ Train/Valid column alignment confirmed
🚀 Cell 6 complete: KNN features computed for VALID set


In [10]:
# ============================================================
# CELL 7: Save KNN-augmented datasets
# ============================================================
# Purpose:
# - Persist KNN-augmented TRAIN/VALID CSVs
# - Keep results reproducible and reusable
# ============================================================

# ---------- 1. Define output directory ----------
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "cHL_CODEX_spatial_knn_features")
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"📁 Output directory: {OUTPUT_DIR}")

# ---------- 2. Define output paths ----------
TRAIN_OUT_PATH = os.path.join(OUTPUT_DIR, "train.csv")
VALID_OUT_PATH = os.path.join(OUTPUT_DIR, "valid.csv")
CLASS_OUT_PATH = os.path.join(OUTPUT_DIR, "class_names.csv")

# ---------- 3. Save CSVs ----------
train_knn_df.to_csv(TRAIN_OUT_PATH, index=False)
valid_knn_df.to_csv(VALID_OUT_PATH, index=False)

# Copy class names unchanged for consistency
class_names.to_csv(CLASS_OUT_PATH, index=False)

print("💾 Saved files:")
print("  -", TRAIN_OUT_PATH)
print("  -", VALID_OUT_PATH)
print("  -", CLASS_OUT_PATH)

# ---------- 4. Verification ----------
# Reload and check shapes
_train_check = pd.read_csv(TRAIN_OUT_PATH)
_valid_check = pd.read_csv(VALID_OUT_PATH)

assert _train_check.shape == train_knn_df.shape, "❌ Train CSV shape mismatch after save"
assert _valid_check.shape == valid_knn_df.shape, "❌ Valid CSV shape mismatch after save"

print("✅ Reload verification passed")

# ---------- 5. Summary ----------
print("\n📐 Feature summary:")
print("  Baseline features :", len(train_df.columns))
print("  KNN features added:", len(PLANNED_KNN_FEATURES))
print("  Total features    :", len(train_knn_df.columns))

print("🚀 Cell 7 complete: KNN-augmented datasets saved")


📁 Output directory: /content/drive/MyDrive/MAPS-update-main/cHL_CODEX_spatial_knn_features
💾 Saved files:
  - /content/drive/MyDrive/MAPS-update-main/cHL_CODEX_spatial_knn_features/train.csv
  - /content/drive/MyDrive/MAPS-update-main/cHL_CODEX_spatial_knn_features/valid.csv
  - /content/drive/MyDrive/MAPS-update-main/cHL_CODEX_spatial_knn_features/class_names.csv
✅ Reload verification passed

📐 Feature summary:
  Baseline features : 55
  KNN features added: 7
  Total features    : 62
🚀 Cell 7 complete: KNN-augmented datasets saved


In [11]:
# ============================================================
# CELL 7.5: Make MAPS importable in Colab
# ============================================================
# Purpose:
# - Ensure original MAPS repo is discoverable by Python
# - No pip install, no runtime restart
# ============================================================

import sys
import os

MAPS_REPO_PATH = "/content/MAPS"

assert os.path.exists(MAPS_REPO_PATH), (
    "❌ MAPS repo not found at /content/MAPS\n"
    "Run CELL 0 (git clone MAPS) first."
)

# Add MAPS repo to Python path
if MAPS_REPO_PATH not in sys.path:
    sys.path.insert(0, MAPS_REPO_PATH)

print("📦 MAPS repo added to sys.path")

# Test imports
try:
    from maps.cell_phenotyping.trainer import Trainer
    from maps.cell_phenotyping.datasets import CellExpressionCSV
    print("✅ MAPS imports successful")
except Exception as e:
    print("❌ MAPS import failed")
    raise e


📦 MAPS repo added to sys.path
✅ MAPS imports successful


In [12]:
# ============================================================
# CELL 8: Baseline MLP training (repo-aligned)
# ============================================================
# Uses:
# - CellExpressionCSV (dataset utility)
# - Custom MLP (same style as teammate)
# - Manual train / validate loop
# ============================================================

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
import tempfile
import os # Import os module to set environment variable

# Disable torch.compile to avoid Triton-related compatibility issues
# This is a workaround for the 'AttributeError: module 'triton.backends' has no attribute 'compiler''
os.environ['TORCH_COMPILE_DISABLE'] = '1'

# ---------- 1. Paths ----------
TRAIN_CSV_ORIGINAL = os.path.join(DATA_SPATIAL, "train.csv")
VALID_CSV_ORIGINAL = os.path.join(DATA_SPATIAL, "valid.csv")

assert os.path.exists(TRAIN_CSV_ORIGINAL)
assert os.path.exists(VALID_CSV_ORIGINAL)

print("📂 Original Baseline CSVs:")
print(" ", TRAIN_CSV_ORIGINAL)
print(" ", VALID_CSV_ORIGINAL)

# ---------- 1.5. Filter DataFrames to use only BASE_FEATURES + LABEL_COL ----------
# The CellExpressionCSV class by default takes all columns except LABEL_COL as features.
# To ensure it only uses BASE_FEATURES, we create filtered temporary CSVs.

train_df_filtered = train_df[BASE_FEATURES + [LABEL_COL]]
valid_df_filtered = valid_df[BASE_FEATURES + [LABEL_COL]]

with tempfile.NamedTemporaryFile(suffix=".csv", delete=False) as tmp_train_file:
    train_df_filtered.to_csv(tmp_train_file.name, index=False)
    TRAIN_CSV = tmp_train_file.name

with tempfile.NamedTemporaryFile(suffix=".csv", delete=False) as tmp_valid_file:
    valid_df_filtered.to_csv(tmp_valid_file.name, index=False)
    VALID_CSV = tmp_valid_file.name

print("📂 Filtered Temp CSVs for dataset loading:")
print(" ", TRAIN_CSV)
print(" ", VALID_CSV)

# ---------- 2. Dataset + DataLoader ----------
from maps.cell_phenotyping.datasets import CellExpressionCSV

BATCH_SIZE = 256
NUM_WORKERS = 2

train_dataset = CellExpressionCSV(TRAIN_CSV, is_train=True)
valid_dataset = CellExpressionCSV(
    VALID_CSV,
    is_train=False,
    mean=train_dataset.mean,
    std=train_dataset.std
)

train_loader = CellExpressionCSV.get_data_loader(
    train_dataset,
    batch_size=BATCH_SIZE,
    is_train=True,
    num_workers=NUM_WORKERS
)

valid_loader = CellExpressionCSV.get_data_loader(
    valid_dataset,
    batch_size=BATCH_SIZE,
    is_train=False,
    num_workers=NUM_WORKERS
)

print(f"📊 Train samples: {len(train_dataset)}")
print(f"📊 Valid samples: {len(valid_dataset)}")

# ---------- 3. Model ----------
from maps.cell_phenotyping.networks import MLP

# Manually define NUM_FEATURES and NUM_CLASSES based on previous cells
# This now correctly aligns with the filtered dataframes.
NUM_FEATURES = len(BASE_FEATURES)
NUM_CLASSES  = len(class_names)

model = MLP(
    input_dim=NUM_FEATURES,
    hidden_dim=512,
    num_classes=NUM_CLASSES,
    dropout=0.10
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"🧠 Model: MLP (6-layer implicit)")
print(f"🧮 Input dim: {NUM_FEATURES}, Classes: {NUM_CLASSES}")

# ---------- 4. Optimizer & Loss ----------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# ---------- 5. Training loop ----------
MAX_EPOCHS = 50
PATIENCE = 5

best_val_loss = float("inf")
patience_counter = 0

for epoch in range(1, MAX_EPOCHS + 1):
    # ---- Train ----
    model.train()
    train_loss = 0.0

    for x, y in tqdm(train_loader, desc=f"Epoch {epoch} [Train]", leave=False):
        x, y = x.to(device), y.to(device)
        x = x.to(torch.float32) # Ensure input is float32

        optimizer.zero_grad()
        logits, _ = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    # ---- Validate ----
    model.eval()
    val_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for x, y in tqdm(valid_loader, desc=f"Epoch {epoch} [Valid]", leave=False):
            x, y = x.to(device), y.to(device)
            x = x.to(torch.float32) # Ensure input is float32
            logits, _ = model(x)
            loss = criterion(logits, y)

            val_loss += loss.item()
            all_preds.append(logits.argmax(dim=1).cpu())
            all_labels.append(y.cpu())

    val_loss /= len(valid_loader)
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    val_acc = accuracy_score(all_labels, all_preds)
    val_f1  = f1_score(all_labels, all_preds, average="macro")

    print(
        f"Epoch {epoch:02d} | "
        f"Train Loss: {train_loss:.4f} | "
        f"Val Loss: {val_loss:.4f} | "
        f"Val Acc: {val_acc:.4f} | "
        f"Val F1: {val_f1:.4f}"
    )

    # ---- Early stopping ----
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_state = model.state_dict()
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print("⏹️ Early stopping triggered")
            break

# ---------- 6. Load best model ----------
model.load_state_dict(best_state)
print("✅ Best model loaded")

# ---------- 7. Final evaluation ----------
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for x, y in valid_loader:
        x, y = x.to(device), y.to(device)
        x = x.to(torch.float32) # Ensure input is float32
        logits, _ = model(x)
        all_preds.append(logits.argmax(dim=1).cpu())
        all_labels.append(y.cpu())

all_preds = torch.cat(all_preds)
all_labels = torch.cat(all_labels)

baseline_acc = accuracy_score(all_labels, all_preds)
baseline_f1  = f1_score(all_labels, all_preds, average="macro")

print("\n📈 BASELINE RESULTS")
print(f"Accuracy : {baseline_acc:.4f}")
print(f"Macro F1 : {baseline_f1:.4f}")

print("🚀 Cell 8 complete: baseline MLP trained")

# ---------- 8. Clean up temporary CSVs ----------
os.remove(TRAIN_CSV)
os.remove(VALID_CSV)
print("🧹 Cleaned up temporary CSV files.")

📂 Original Baseline CSVs:
  /content/drive/MyDrive/MAPS-update-main/cHL_CODEX_spatial_features/train.csv
  /content/drive/MyDrive/MAPS-update-main/cHL_CODEX_spatial_features/valid.csv
📂 Filtered Temp CSVs for dataset loading:
  /tmp/tmpt7k32uis.csv
  /tmp/tmpza5p2y6l.csv
📊 Train samples: 114984
📊 Valid samples: 28746
🧠 Model: MLP (6-layer implicit)
🧮 Input dim: 52, Classes: 16


Epoch 01 | Train Loss: 2.6698 | Val Loss: 2.2550 | Val Acc: 0.2590 | Val F1: 0.2263


Epoch 02 | Train Loss: 1.9804 | Val Loss: 1.9638 | Val Acc: 0.3568 | Val F1: 0.3345


Epoch 03 | Train Loss: 1.7923 | Val Loss: 1.8624 | Val Acc: 0.3615 | Val F1: 0.3569


Epoch 04 | Train Loss: 1.5749 | Val Loss: 1.5394 | Val Acc: 0.4547 | Val F1: 0.4624


...

Epoch 47 | Train Loss: 0.4637 | Val Loss: 0.6303 | Val Acc: 0.7727 | Val F1: 0.7655


Epoch 48 | Train Loss: 0.4691 | Val Loss: 0.5912 | Val Acc: 0.7877 | Val F1: 0.7779


Epoch 49 | Train Loss: 0.4598 | Val Loss: 0.5878 | Val Acc: 0.7859 | Val F1: 0.7764


Epoch 50 | Train Loss: 0.4603 | Val Loss: 0.5827 | Val Acc: 0.7892 | Val F1: 0.7801
✅ Best model loaded



📈 BASELINE RESULTS
Accuracy : 0.7892
Macro F1 : 0.7801
🚀 Cell 8 complete: baseline MLP trained
🧹 Cleaned up temporary CSV files.


CELL 9 — KNN-Augmented Baseline (Spatial Features → MLP)

Goal:
Create KNN spatial summary features → append to input → retrain same MLP → compare against baseline (0.7892 / 0.7801).

No architecture changes. No optimizer changes.

In [13]:
# ============================================================
# CELL 9.1: KNN feature configuration
# ============================================================

from sklearn.neighbors import NearestNeighbors
import numpy as np

K = 5  # spatial neighbors (keep fixed for first experiment)

# Choose a SMALL, biologically meaningful subset
KNN_MARKERS = [
    "CD4",
    "CD8",
    "CD20",
    "CD68",
    "CD45"
]

print("🔢 KNN configuration")
print("K =", K)
print("Markers used for KNN summaries:", KNN_MARKERS)


🔢 KNN configuration
K = 5
Markers used for KNN summaries: ['CD4', 'CD8', 'CD20', 'CD68', 'CD45']


In [14]:
# ============================================================
# CELL 9.2: Compute KNN spatial features
# ============================================================

def add_knn_features(df, k, marker_cols):
    coords = df[["X_cent", "Y_cent"]].values

    knn = NearestNeighbors(n_neighbors=k + 1, metric="euclidean")
    knn.fit(coords)

    distances, indices = knn.kneighbors(coords)

    # Drop self-neighbor
    distances = distances[:, 1:]
    indices = indices[:, 1:]

    # Distance features
    df["knn_mean_dist"] = distances.mean(axis=1)
    df["knn_std_dist"]  = distances.std(axis=1)

    # Marker neighborhood means
    for m in marker_cols:
        df[f"knn_mean_{m}"] = np.array([
            df.iloc[neighbors][m].mean()
            for neighbors in indices
        ])

    return df


In [15]:
# ============================================================
# CELL 9.3: Apply KNN features
# ============================================================

train_df_knn = add_knn_features(train_df.copy(), K, KNN_MARKERS)
valid_df_knn = add_knn_features(valid_df.copy(), K, KNN_MARKERS)

KNN_FEATURES = (
    ["knn_mean_dist", "knn_std_dist"] +
    [f"knn_mean_{m}" for m in KNN_MARKERS]
)

print("🧬 KNN features added:")
for f in KNN_FEATURES:
    print(" ", f)

print("Total new features:", len(KNN_FEATURES))


🧬 KNN features added:
  knn_mean_dist
  knn_std_dist
  knn_mean_CD4
  knn_mean_CD8
  knn_mean_CD20
  knn_mean_CD68
  knn_mean_CD45
Total new features: 7


In [16]:
# ============================================================
# CELL 9.4: Create KNN-augmented CSVs for training
# ============================================================

import tempfile

FEATURES_WITH_KNN = BASE_FEATURES + KNN_FEATURES

train_df_knn_filtered = train_df_knn[FEATURES_WITH_KNN + [LABEL_COL]]
valid_df_knn_filtered = valid_df_knn[FEATURES_WITH_KNN + [LABEL_COL]]

with tempfile.NamedTemporaryFile(suffix=".csv", delete=False) as tmp_train:
    train_df_knn_filtered.to_csv(tmp_train.name, index=False)
    TRAIN_CSV_KNN = tmp_train.name

with tempfile.NamedTemporaryFile(suffix=".csv", delete=False) as tmp_valid:
    valid_df_knn_filtered.to_csv(tmp_valid.name, index=False)
    VALID_CSV_KNN = tmp_valid.name

print("📂 KNN-augmented CSVs:")
print(" ", TRAIN_CSV_KNN)
print(" ", VALID_CSV_KNN)
print("🧮 Total input features:", len(FEATURES_WITH_KNN))


📂 KNN-augmented CSVs:
  /tmp/tmpd6cawyw8.csv
  /tmp/tmpcjd29v1o.csv
🧮 Total input features: 59


In [17]:
# ============================================================
# CELL 9.5: KNN-augmented MLP training (same as baseline)
# ============================================================

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
import tempfile
import os

# Disable torch.compile to avoid Triton-related issues (same as Cell 8)
os.environ['TORCH_COMPILE_DISABLE'] = '1'

# ---------- 1. Create filtered temp CSVs (BASE + KNN features) ----------
train_df_knn_filtered = train_df_knn[FEATURES_WITH_KNN + [LABEL_COL]]
valid_df_knn_filtered = valid_df_knn[FEATURES_WITH_KNN + [LABEL_COL]]

with tempfile.NamedTemporaryFile(suffix=".csv", delete=False) as tmp_train:
    train_df_knn_filtered.to_csv(tmp_train.name, index=False)
    TRAIN_CSV_KNN = tmp_train.name

with tempfile.NamedTemporaryFile(suffix=".csv", delete=False) as tmp_valid:
    valid_df_knn_filtered.to_csv(tmp_valid.name, index=False)
    VALID_CSV_KNN = tmp_valid.name

print("📂 KNN-augmented temp CSVs:")
print(" ", TRAIN_CSV_KNN)
print(" ", VALID_CSV_KNN)

# ---------- 2. Dataset + DataLoader ----------
from maps.cell_phenotyping.datasets import CellExpressionCSV

BATCH_SIZE = 256
NUM_WORKERS = 2

train_dataset = CellExpressionCSV(TRAIN_CSV_KNN, is_train=True)
valid_dataset = CellExpressionCSV(
    VALID_CSV_KNN,
    is_train=False,
    mean=train_dataset.mean,
    std=train_dataset.std
)

train_loader = CellExpressionCSV.get_data_loader(
    train_dataset,
    batch_size=BATCH_SIZE,
    is_train=True,
    num_workers=NUM_WORKERS
)

valid_loader = CellExpressionCSV.get_data_loader(
    valid_dataset,
    batch_size=BATCH_SIZE,
    is_train=False,
    num_workers=NUM_WORKERS
)

print(f"📊 Train samples: {len(train_dataset)}")
print(f"📊 Valid samples: {len(valid_dataset)}")

# ---------- 3. Model ----------
from maps.cell_phenotyping.networks import MLP

NUM_FEATURES = len(FEATURES_WITH_KNN)
NUM_CLASSES  = len(class_names)

model = MLP(
    input_dim=NUM_FEATURES,
    hidden_dim=512,
    num_classes=NUM_CLASSES,
    dropout=0.10
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("🧠 Model: MLP (KNN-augmented)")
print(f"🧮 Input dim: {NUM_FEATURES}, Classes: {NUM_CLASSES}")

# ---------- 4. Optimizer & Loss ----------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# ---------- 5. Training loop ----------
MAX_EPOCHS = 50
PATIENCE = 5

best_val_loss = float("inf")
patience_counter = 0

for epoch in range(1, MAX_EPOCHS + 1):
    # ---- Train ----
    model.train()
    train_loss = 0.0

    for x, y in tqdm(train_loader, desc=f"Epoch {epoch} [Train]", leave=False):
        x, y = x.to(device), y.to(device)
        x = x.to(torch.float32)

        optimizer.zero_grad()
        logits, _ = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    # ---- Validate ----
    model.eval()
    val_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for x, y in tqdm(valid_loader, desc=f"Epoch {epoch} [Valid]", leave=False):
            x, y = x.to(device), y.to(device)
            x = x.to(torch.float32)
            logits, _ = model(x)
            loss = criterion(logits, y)

            val_loss += loss.item()
            all_preds.append(logits.argmax(dim=1).cpu())
            all_labels.append(y.cpu())

    val_loss /= len(valid_loader)
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    val_acc = accuracy_score(all_labels, all_preds)
    val_f1  = f1_score(all_labels, all_preds, average="macro")

    print(
        f"Epoch {epoch:02d} | "
        f"Train Loss: {train_loss:.4f} | "
        f"Val Loss: {val_loss:.4f} | "
        f"Val Acc: {val_acc:.4f} | "
        f"Val F1: {val_f1:.4f}"
    )

    # ---- Early stopping ----
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        best_state = model.state_dict()
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print("⏹️ Early stopping triggered")
            break

# ---------- 6. Load best model ----------
model.load_state_dict(best_state)
print("✅ Best KNN model loaded")

# ---------- 7. Final evaluation ----------
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for x, y in valid_loader:
        x, y = x.to(device), y.to(device)
        x = x.to(torch.float32)
        logits, _ = model(x)
        all_preds.append(logits.argmax(dim=1).cpu())
        all_labels.append(y.cpu())

all_preds = torch.cat(all_preds)
all_labels = torch.cat(all_labels)

knn_acc = accuracy_score(all_labels, all_preds)
knn_f1  = f1_score(all_labels, all_preds, average="macro")

print("\n📈 KNN-AUGMENTED RESULTS")
print(f"Accuracy : {knn_acc:.4f}")
print(f"Macro F1 : {knn_f1:.4f}")

# ---------- 8. Cleanup ----------
os.remove(TRAIN_CSV_KNN)
os.remove(VALID_CSV_KNN)
print("🧹 Temporary KNN CSV files removed")
print("🚀 Cell 9.5 complete: KNN-augmented MLP trained")


📂 KNN-augmented temp CSVs:
  /tmp/tmpa_1mkkr6.csv
  /tmp/tmpj61yn2kh.csv
📊 Train samples: 114984
📊 Valid samples: 28746
🧠 Model: MLP (KNN-augmented)
🧮 Input dim: 59, Classes: 16


Epoch 01 | Train Loss: 2.6680 | Val Loss: 2.3418 | Val Acc: 0.1123 | Val F1: 0.0816


Epoch 02 | Train Loss: 2.1554 | Val Loss: 2.0069 | Val Acc: 0.3032 | Val F1: 0.2988


Epoch 03 | Train Loss: 1.8000 | Val Loss: 1.8242 | Val Acc: 0.3654 | Val F1: 0.3470


Epoch 04 | Train Loss: 1.5706 | Val Loss: 1.6109 | Val Acc: 0.4223 | Val F1: 0.4194


...

Epoch 40 | Train Loss: 0.5103 | Val Loss: 0.6514 | Val Acc: 0.7719 | Val F1: 0.7619


Epoch 41 | Train Loss: 0.5006 | Val Loss: 0.6408 | Val Acc: 0.7731 | Val F1: 0.7607


Epoch 42 | Train Loss: 0.4978 | Val Loss: 0.6573 | Val Acc: 0.7698 | Val F1: 0.7611


Epoch 43 | Train Loss: 0.4845 | Val Loss: 0.6400 | Val Acc: 0.7761 | Val F1: 0.7678
⏹️ Early stopping triggered
✅ Best KNN model loaded



📈 KNN-AUGMENTED RESULTS
Accuracy : 0.7761
Macro F1 : 0.7678
🧹 Temporary KNN CSV files removed
🚀 Cell 9.5 complete: KNN-augmented MLP trained
